In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformer import TransformerConfig
from lm import LM

from data import Dataset

In [2]:
dataset = Dataset()

config = TransformerConfig(d_model=128, n_layers=1, n_heads=4, max_len=dataset.max_len, dropout=0.)
model = LM(config, vocab_size=len(dataset.vocabulaire))
model.load_state_dict(torch.load("transformer.pth"))

<All keys matched successfully>

In [7]:
def sample(self, prompt = "", g = torch.Generator()):
    idx = torch.tensor([dataset.char_to_int[c] for c in prompt], dtype=torch.int32).unsqueeze(0)
    idx = torch.cat([torch.tensor(dataset.char_to_int['<SOS>']).view(1, 1), idx], dim=1)
    next_id = -1

    while next_id != dataset.char_to_int['<EOS>']:
        act = self(idx, act=True) # (1, l, d_model)
        #act[:, :, 56] = 3*0.8

        x = self.out_norm(act)
        logits = self.lm_head(x)

        probs = F.softmax(logits[:, -1, :], dim=-1)
        next_id = torch.multinomial(probs, num_samples=1, generator=g).item()
        idx = torch.cat([idx, torch.tensor(next_id).view(1, 1)], dim=1)

    #print(act.max(dim=2).indices)
    #print(act[:, :, 21])
        
    return "".join([dataset.int_to_char[p.item()] for p in idx[0, 1:-1]])

In [17]:
for i in range(10):
    print(sample(model, ""))

vaudolles
vadampart-sur-vieux
augenols
saint-julnier-les-galandeumois-le
marey-en-de-burandon
fron
vérans-en-madeau
la chemaule
carlos
la hacré
